Exploring LangChain
https://python.langchain.com/en/latest/getting_started/getting_started.html

In [1]:
### RUN ONCE
### ! pip install openai ### use conda below
# ! conda install langchain -c conda-forge --yes

In [2]:
import os
from langchain.llms import OpenAI

In [3]:
# ! echo $OPENAI_API_KEY
#import os; # os.getenv('OPENAI_API_KEY', 'DEFAULT_VALUE') # print(os.environ)
env_OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', 'DEFAULT_VALUE')
# print(env_OPENAI_API_KEY)

In [4]:
llm = OpenAI(openai_api_key=env_OPENAI_API_KEY) # works without the key, too

In [5]:
llm = OpenAI(temperature=0.6)

In [6]:
text = "What would be a good company name for a company that makes colorful socks?"
# Cheery Toes. # temperature=0.8
# Rainbow Socks Co. # temperature=0.1
print(llm(text))



BrightSteps Socks.


# PromptTemplate

In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [8]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


# Chains: Combine LLMs and prompts in multi-step workflows
https://python.langchain.com/en/latest/getting_started/getting_started.html#chains-combine-llms-and-prompts-in-multi-step-workflows

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
chain.run("colorful socks")

'\n\nRainbow Socks Co.'

https://python.langchain.com/en/latest/modules/chains/getting_started.html

# Agents: Dynamically Call Chains Based on User Input
https://python.langchain.com/en/latest/getting_started/getting_started.html#agents-dynamically-call-chains-based-on-user-input

In [11]:
# ! conda install google-search-results -c conda-forge --yes # PackagesNotFoundError
# ! pip install google-search-results

In [12]:
import os
env_SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY', 'DEFAULT_VALUE')
print(env_SERPAPI_API_KEY)

DEFAULT_VALUE


In [13]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [14]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

## The rest of the section does not work because imports failed

# Callbacks
https://python.langchain.com/en/latest/modules/callbacks/getting_started.html#callbacks


LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.

You can subscribe to these events by using the callbacks argument available throughout the API. This argument is a list of handler objects expected to implement one or more of the methods described below in more detail. There are two main callbacks mechanisms:

Constructor callbacks will be used for all calls made on that object and will be scoped to that object only, i.e., if you pass a handler to the LLMChain constructor, it will not be used by the model attached to that chain.

Request callbacks will be used for that specific request only and all sub-requests it contains (e.g., a call to an LLMChain triggers a call to a Model, which uses the same handler passed through). These are explicitly passed through.

Advanced: 
When you create a custom chain, you can easily set it up to use the same callback system as all the built-in chains. _call, _generate, _run and equivalent async methods on Chains / LLMs / Chat Models / Agents / Tools now receive a 2nd argument called run_manager which is bound to that run and contains the logging methods that can be used by that object (i.e., on_llm_new_token). This is useful when constructing a custom chain. Please take a look at this guide for more information on creating custom chains and using callbacks inside them.

CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

# How to use callbacks
https://python.langchain.com/en/latest/modules/callbacks/getting_started.html#how-to-use-callbacks

The callbacks argument is available on most objects throughout the API (Chains, Models, Tools, Agents, etc.) in two different places:

## Constructor callbacks: 
defined in the constructor, e.g., LLMChain(callbacks=[handler]), which will be used for all calls made on that object and will be scoped to that object only, e.g., if you pass a handler to the LLMChain constructor, it will not be used by the Model attached to that chain.

## Request callbacks: 
defined in the call() run() apply() methods used for issuing a request, e.g. chain.call(inputs, callbacks=[handler]), which will be used for that specific request only, and all sub-requests that it contains (e.g. a call to an LLMChain triggers a call to a Model, which uses the same handler passed in the call() method).

The verbose argument is available on most objects throughout the API (Chains, Models, Tools, Agents, etc.) as a constructor argument, e.g., LLMChain(verbose=True), and it is equivalent to passing a ConsoleCallbackHandler to the callbacks argument of that object and all child objects. This is useful for debugging, as it will log all events to the console.

## When do you want to use each of these?
Constructor callbacks are most useful for use cases such as logging, monitoring, etc., which are not specific to a single request but to the entire chain. 
For example, if you want to log all the requests made to an LLMChain, you would pass a handler to the constructor.

Request callbacks are most useful for use cases such as streaming, where you want to stream the output of a single request to a specific web socket connection, or other similar use cases. For example, if you want to stream the output of a single request to a websocket, you would pass a handler to the call() method

## Using an existing handler
LangChain provides a few built-in handlers that you can use to get started. These are available in the long chain/callbacks module. The most basic handler is the StdOutCallbackHandler, which logs all events to stdout. In the future, we will add more default handlers to the library.

Note when the verbose flag on the object is set to true, the StdOutCallbackHandler will be invoked even without being explicitly passed in.

In [19]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

In [24]:
# First, let's explicitly set the StdOutCallbackHandler in `callbacks`
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n\n3'

In [25]:
# Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n\n3'

In [22]:
# Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n3'